# Submitting Warnings for Mercury Challenge Using Python

This notebook will walk through the process of submitting Mercury Challenge warnings using the Python's **Requests** library.  In order to submit warnings a challenge participant must be registered with TopCoder and be in possession of an API key, which will be used to authenticate submissions.  For more information on this please see https://iarpa.gov/challenges/mercury.html and TopCoder's instructions at https://docs.google.com/document/d/1KViMdqLFecnjEur6pE6-fHKcoO94PkK1g7XWwpEyCOE/edit

In this notebook we will use the fictional participant name "testuser" who has obtained the API key "mysecretkey" after registering for the challenge.

## Warning Format

All submissions for the Mercury Challenge must be valid JSON and must contain the fields described in the Challenge Handbook https://github.com/planetmercury/mercury-challenge/blob/master/doc/handbook/Mercury%20Challenge%20-%20Handbook.pdf.  Missing or extraneous submissions will likely result in the rejection of your submission.

The submission has two top level fields, `participant_id` and `payload`.  The `participant_id` must match the API Key.  The `payload` must be a list of JSON-formatted warnings.

Example:

```
{
  "participant_id": "testuser",
  "payload": [
    {
      "Actor": "Royal Saudi Military",
      "City": "Al Qaţīf",
      "Event_Date": "2018-06-17",
      "Event_Subtype": "Conflict",
      "Latitude": 26.5208,
      "Longitude": 50.0245,
      "State": "Eastern Province",
      "Country": "Saudi Arabia",
      "Event_Type": "Military Activity",
      "Warning_ID": "3148a892c3a145dc87df59532018100e"
    },
    {
      "Actor": "Royal Saudi Military",
      "City": "Al ‘Awāmīyah",
      "Event_Date": "2018-06-22",
      "Event_Subtype": "Conflict",
      "Latitude": 26.5936,
      "Longitude": 49.9875,
      "State": "Eastern Province",
      "Country": "Saudi Arabia",
      "Event_Type": "Military Activity",
      "Warning_ID": "0736f9dd320e431d95007faad74c0d05"
    }
  ]
}
```

In this example the `testuser` participant has prepared a payload of two Military Activity warnings for Saudi Arabia.

The submission URL for the challenge is https://87g554i96c.execute-api.us-east-1.amazonaws.com/Production/warning/intake.  Submissions must use **POST** and must have the following headers:
- "Content-Type:application/json"
- "x-api-key:TopCoderusername:token"

Note that you will not be able to run these commands directly from this notebook without modification.  You will need to use your own user name and token and you'll need to modify the warning content.  Please also note that this is the production endpoint, so any submissions will be scored.  

In this example, the JSON content is in a file named "warnfile.json" in the current working directory.

## Using Python

In [1]:
import requests
from uuid import uuid4 #Used to generate unique warning IDs
from pprint import pprint

secret_token = "mysecretkey"
userid = "testuser"
url = "https://87g554i96c.execute-api.us-east-1.amazonaws.com/Production/warning/intake"
headers = {'x-api-key':'{0}:{1}'.format(userid, secret_token)}

Below we build a Python **dict** for our submission. The keys for this dict will be the `participant_id` and `payload`.  The `payload` will be a list of dicts, where the keys to these dict correspond to the fields in each warning.  In order to generate unique warning identifiers we use the uuid4() method and append it to the participant ID.

(By the way, please note that the example below is very similar to the example above but it has been shifted into August 2019.)

In [2]:
submission = {"participant_id": userid,
              "payload": []}
country = "Saudi Arabia"
event_type = "Military Activity"
subtype = "Conflict"
warn_id_0 = "{0}{1}".format(userid, uuid4().hex)
event_date_0 = "2019-08-22"
actor_0 = "Royal Saudi Military"
lat_0 = 26.5208
lon_0 = 50.0245
city_0 = "Al Qaţīf"
state_0 = "Eastern Province"
warn_content_0 = {"Actor": actor_0,
                "City": city_0,
                "State": state_0,
                "Country": country,
                "Latitude": lat_0,
                "Longitude": lon_0,
                "Event_Date": event_date_0,
                "Event_Type": event_type,
                "Event_Subtype": subtype,
                "Warning_ID": warn_id_0}
warn_id_1 = "{0}{1}".format(userid, uuid4().hex)
event_date_1 = "2019-08-24"
actor_1 = "Royal Saudi Military"
lat_1 = 26.5936
lon_1 = 49.9875
city_1 = "Al ‘Awāmīyah"
state_1 = "Eastern Province"
warn_content_1 = {"Actor": actor_1,
                "City": city_1,
                "State": state_1,
                "Country": country,
                "Latitude": lat_1,
                "Longitude": lon_1,
                "Event_Date": event_date_1,
                "Event_Type": event_type,
                "Event_Subtype": subtype,
                "Warning_ID": warn_id_1}

submission["payload"] = [warn_content_0, warn_content_1]

pprint(submission)

{'participant_id': 'testuser',
 'payload': [{'Actor': 'Royal Saudi Military',
              'City': 'Al Qaţīf',
              'Country': 'Saudi Arabia',
              'Event_Date': '2019-08-22',
              'Event_Subtype': 'Conflict',
              'Event_Type': 'Military Activity',
              'Latitude': 26.5208,
              'Longitude': 50.0245,
              'State': 'Eastern Province',
              'Warning_ID': 'testuser150f4133679d4448aaa2174c945534bd'},
             {'Actor': 'Royal Saudi Military',
              'City': 'Al ‘Awāmīyah',
              'Country': 'Saudi Arabia',
              'Event_Date': '2019-08-24',
              'Event_Subtype': 'Conflict',
              'Event_Type': 'Military Activity',
              'Latitude': 26.5936,
              'Longitude': 49.9875,
              'State': 'Eastern Province',
              'Warning_ID': 'testuserc392c27b6e3b4983925feeaa6c90dd68'}]}


Using the `post` method for requests we make our submission.  We can then examine the `json` attribute of the returned results to see if our submission was successful.

In [23]:
result = requests.post(url, headers=headers, json=submission)

pprint(result.json())

{'Existing warnings updated': 0,
 'Invalid warnings': 0,
 'New warnings or versions inserted': 2,
 'Total operations': 2,
 'reason': None,
 'result': 'ok'}


## Results

If your submission is successful you will see an output like this

If your submission is unsuccessful you will receive an error message.  Common error messages and their meanings are listed below.  Please note that some error messages only pertain to some of the warnings in your payload; the other content in your payload will have been successfully submitted.

This means that the `participant_id` in your submission content doesn't match the participant id in your header.

This, and similar "Cannot update..." messages indicate that you tried to change an immutable field in your warning.  In this case the new version of the warning changed the `Country`, which is not allowed.

This indicates an invalid event type was used.

This indicates a missing required field.

This indicates that you are submitting warnings too frequently.  Wait the required length of time and try again.

In this case a negative value for `Case_Count` was attempted.